In [19]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score

In [3]:
data = pd.read_csv("../data/clean/loans_train_test.csv", sep = "^")

In [4]:
data.head()

,funded_amnt_inv,issue_d,installment,int_rate,annual_inc,dti,last_pymnt_amnt,total_pymnt_inv,total_rec_late_fee,total_acc,...,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,application_type_Individual,application_type_Joint App
0,10000.0,2016,262.34,19.53,52000.0,15.00,9575.49,11127.83,0.0,12.0,...,0,0,0,0,0,0,0,0,1,0
1,35000.0,2016,941.96,20.75,85000.0,24.98,509.10,37226.47,0.0,19.0,...,0,0,0,0,0,0,0,0,1,0
2,14400.0,2016,469.74,10.75,85000.0,28.11,10992.93,15673.13,0.0,33.0,...,0,0,0,0,0,0,0,0,1,0
3,18000.0,2016,400.31,11.99,100000.0,20.41,14885.49,20465.85,0.0,40.0,...,0,0,0,0,1,0,0,0,1,0
4,7250.0,2016,246.63,13.67,72000.0,23.93,6310.66,7779.43,0.0,24.0,...,0,0,0,0,0,0,0,0,1,0


Train / Test split:

In [7]:
X = data.loc[:, data.columns!='loan_status']

In [8]:
y = data['loan_status']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Classifiers:

In [14]:
logit_cl = LogisticRegression(random_state=42)

In [13]:
random_forest = RandomForestClassifier(random_state=42)

In [17]:
#Cross validate model with Kfold stratified cross val
kfold = StratifiedKFold(n_splits=10)

In [20]:
cross_val_score(logit_cl, X_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4)

array([0.98401626, 0.98420362, 0.9847369 , 0.98377124, 0.98478013,
       0.9838433 , 0.98395838, 0.98384284, 0.98398697, 0.98424641])

Prior:

In [25]:
y.value_counts()[0] / len(y)

0.7891647324563376

In [26]:
cross_val_score(logit_cl, X_train, y = y_train, scoring = "roc_auc", cv = kfold, n_jobs=4)

array([0.99314456, 0.99178011, 0.99268939, 0.99263278, 0.99220848,
       0.99288253, 0.9928652 , 0.99255636, 0.9923361 , 0.99276589])

In [27]:
cross_val_score(random_forest, X_train, y = y_train, scoring = "roc_auc", cv = kfold, n_jobs=4)

array([0.99518351, 0.99519057, 0.99543128, 0.99593549, 0.99513263,
       0.99511966, 0.99547759, 0.99556798, 0.99544845, 0.9950772 ])